In [ ]:
import numpy as np
import os
import tarfile
import sys
import tensorflow as tf
import zipfile
import six.moves.urllib as urllib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash
cd models/research/
pip install .

In [ ]:
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
!pip install kaggle

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [ ]:
from .kaggle import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle datasets download -d smeschke/four-shapes
!ls

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
#datashapes = pd.read_csv('/content/four-shapes.zip', compression='zip', header=0, sep=',', quotechar='"')
!unzip four-shapes.zip -d input

In [ ]:
import os 

MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11-17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

PATH_TO_CKPT = MODEL_NAME + 'frozen_inference_graph.pb'

PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 4

In [ ]:
def load_model(model_name):
  #base_url = 'http://download.tensorflow.org/models/object_detection/'
  base_url = '/home/cathalina/deeplearning/Chapter4/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
      fname = model_name,
      origin = base_url + model_file,
      untar = True
  )

  model = tf.saved_model.load(str(model_dir), tags=None)
  model = model.signatures['serving_default']

  return model

In [ ]:
#from object_detection.utils import label_map_util

PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name = True)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGES_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob('*.jpg')))
TEST_IMAGES_PATHS

In [ ]:
model_name = ('ssd_mobilenet_v1_coco_2017_11_17')
detection_model = load_model(model_name)

In [ ]:
print(detection_model.inputs)

In [ ]:
detection_model.output_dtypes

In [ ]:
detection_models.output_shapes

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  output_dict = model(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() for key, value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
        output_dict['detection_masks'], output_dict['detection_boxes'], image.shape[0], image.shape[1]
    )
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

In [ ]:
def show_inference(model, image_path):
  image_np = np.array(Image.open(image_path))
  output_dict = run_inference_for_single_image(model, image_np)

  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks = output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates = True,
      line_thickness = 8
  )

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGES_PATHS:
  show_inference(detection_model, image_path
                 )

In [ ]:
#segmentation

model_name = 'mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'
masking_model = load_model('mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28')

In [ ]:
masking_model.output_shapes

In [ ]:
for image_path in TEST_IMAGES_PATHS:
  show_inference(masking_model, image_path)